In [3]:
example1 = list(map(int, """
16
10
15
5
1
11
7
19
6
12
4
""".strip().splitlines()))

example2 = list(map(int, """
28
33
18
42
31
14
46
20
48
47
24
23
49
45
19
38
39
11
1
32
25
35
8
17
7
9
4
2
34
10
3
""".strip().splitlines()))

with open("day10.txt", "r") as f:
    data = list(map(int, f.readlines()))

In [4]:
from typing import List, Tuple

def get_chain_differences(adapters: List[int]) -> Tuple[int, int, int]:
    adapters.sort()

    jolts = 0
    counts = [0, 0, 0]
    for adapter in adapters:
        if adapter > jolts + 3:
            return counts

        counts[adapter - jolts - 1] += 1
        jolts = adapter

    return (counts[0], counts[1], counts[2] + 1)

print(f"Differences for example1: {get_chain_differences(example1)}")
assert get_chain_differences(example1) == (7, 0, 5)


print(f"Differences for example2: {get_chain_differences(example2)}")
assert get_chain_differences(example2) == (22, 0, 10)

real_differences = get_chain_differences(data)
print(f"Differences for real data: {real_differences} ({real_differences[0] * real_differences[2]})")

Differences for example1: (7, 0, 5)
Differences for example2: (22, 0, 10)
Differences for real data: (72, 0, 31) (2232)


## Part 2 Algorithm
We can visualize the sequence of adapters (and the differences between them) as a series of connected springs.
Segments with a difference of `3` cannot be separated further without breaking the springs, however segments with
a difference of `1` can be extended to span `2` or `3` places by removing intermediate segments.


For example, if we have the following sequence of differences:

```
[3, 1, 3, 3, 1, 3]
```

We know that our valid combinations are as follows:

```
[3, 1, 3, 3, 1, 3]
[3, 1, 3, 3, 2, 3]
[3, 1, 3, 3, 3, 3]
[3, 2, 3, 3, 1, 3]
[3, 2, 3, 3, 2, 3]
[3, 2, 3, 3, 3, 3]
[3, 3, 3, 3, 1, 3]
[3, 3, 3, 3, 2, 3]
[3, 3, 3, 3, 3, 3]
```

So in this case, we have 9 total combinations.

We can use a dynamic programming approach here to, starting at one end of the sequence, start varying the number of
removed segments and move our way back to the start. Each time we vary the number of removed segments, the total number
of combinations increases by a factor of the number of combinations which preceeded it in the sequence.

If we start at the end of the sequence and count the number of combinations, working back to the front, we get the
following output:

```
seq:   [3, 1, 3, 3, 1, 3]
comb:  [0, 0, 0, 0, 0, 1]
       [0, 0, 0, 0, 3, 1]
       [0, 0, 0, 3, 3, 1]
       [0, 0, 3, 3, 3, 1]
       [0, 9, 3, 3, 3, 1]
       [9, 9, 3, 3, 3, 1]
```

This gives us the same `9` combinations that we saw when manually testing this out and allows us to calculate this in
`O(N)` time using `O(N)` memory. This is plenty fast enough for our dataset.

In [19]:
from typing import Dict 

def get_chain_arrangements(adapters: List[int]) -> int:
    adapters = [0, *adapters]

    combinations = [0 for i in range(len(adapters))]
    combinations[len(adapters)-1] = 1

    for i in range(len(adapters) - 2, -1, -1):
        for j in range(i + 1, len(adapters)):
            if adapters[j] - adapters[i] <= 3:
                combinations[i] += combinations[j]

    return combinations[0]

print(f"Total arrangements for example1: {get_chain_arrangements(example1)}")
assert get_chain_arrangements(example1) == 8

print(f"Total arrangements for example2: {get_chain_arrangements(example2)}")
assert get_chain_arrangements(example2) == 19208

Total arrangements for example1: 8
Total arrangements for example2: 19208


In [20]:
print(f"Total arrangements for real data: {get_chain_arrangements(data)}")

Total arrangements for real data: 173625106649344
